# Values Tutorial

[Values](../api/symforce.values.values.html#symforce.values.values.Values) objects are ordered dict-like containers used to store multiple heterogeneous objects, normally for the purpose of function generation.

Typically a Values object will first be created to defines a number of different symbolic inputs (e.g. rotations, translations, scalars, poses, cameras, etc.). Then, a second Values object will be created to describe the objects to be returned from the function, which will be composed of the symbolic elements defined by the input Values object.

In [ ]:
# Setup
import symforce

symforce.set_symbolic_api("sympy")
symforce.set_log_level("warning")

import symforce.symbolic as sf
from symforce.values import Values
from symforce.notebook_util import display, display_code, display_code_file

In [ ]:
inputs = Values(
    x=sf.Symbol("x"),
    y=sf.Rot2.symbolic("c"),
)
display(inputs)

The `.add()` method can add a symbol using its name as the key:

In [ ]:
inputs.add(sf.Symbol("foo"))
display(inputs)

Adding sub-values are well encouraged:

In [ ]:
x, y = sf.symbols("x y")
expr = x ** 2 + sf.sin(y) / x ** 2
inputs["states"] = Values(p=expr)
display(inputs)

A Values serializes to a depth-first traversed list. This means it implements StorageOps:

In [ ]:
display(inputs.to_storage())

We can also get a flattened lists of keys and values, with `.` separation for sub-values:

In [ ]:
display(inputs.items_recursive())

Note that there is a `.keys_recursive()` and a `.values_recursive()` which return flattened lists of keys and values respectively:

In [ ]:
display(inputs.keys_recursive())
display(inputs.values_recursive())

To fully reconstruct the types in the Values from the serialized scalars, we need an index that describes which parts of the serialized list correspond to which types. The spec is `T.Dict[str, IndexEntry]` where `IndexEntry` has attributes `offset, storage_dim, datatype, shape, item_index`:

In [ ]:
index = inputs.index()
index

With a serialized list and an index, we can get the values back:

In [ ]:
inputs2 = Values.from_storage_index(inputs.to_storage(), index)
assert inputs == inputs2
display(inputs)

The `item_index` is a recursive structure that can contain the index for a sub-values:

In [ ]:
item_index = inputs.index()["states"].item_index
assert item_index == inputs["states"].index()

We can also set sub-values directly with dot notation in the keys. They get split up:

In [ ]:
inputs["states.blah"] = 3
display(inputs)

The `.attr` field also allows attribute access rather than key access:

In [ ]:
assert inputs["states.p"] is inputs["states"]["p"] is inputs.attr.states.p
display(inputs.attr.states.p)

Finally, SymForce adds the concept of a name scope to namespace symbols. Within a scope block, symbol names get prefixed with the scope name:

In [ ]:
with sf.scope("params"):
    s = sf.Symbol("cost")
display(s)

A common use case is to call a function that adds symbols within your own name scope to avoid name collisions. You can also chain name scopes:

In [ ]:
v = Values()
v.add(sf.Symbol("x"))
with sf.scope("foo"):
    v.add(sf.Symbol("x"))
    with sf.scope("bar"):
        v.add(sf.Symbol("x"))
display(v)
display(v.attr.foo.bar.x)

The values class also provides a `.scope()` method that not only applies the scope to symbol names but also to keys added to the Values:

In [ ]:
v = Values()
with v.scope("hello"):
    v["y"] = x ** 2
    v["z"] = sf.Symbol("z")
v

This flexible set of features provided by the Values class allows conveniently building up large expressions, and acts as the interface to code generation.

## Lie Group Operations

One useful feature of Values objects is that element-wise Lie group operations on can be performed on them.

In [ ]:
lie_vals = Values()
lie_vals["scalar"] = sf.Symbol("x")
lie_vals["rot3"] = sf.Rot3.symbolic("rot")

sub_lie_vals = Values()
sub_lie_vals["pose3"] = sf.Pose3.symbolic("pose")
sub_lie_vals["vec"] = sf.V3.symbolic("vec")

lie_vals["sub_vals"] = sub_lie_vals

display(lie_vals)

In [ ]:
display(lie_vals.tangent_dim())
display(len(lie_vals.to_tangent()))

Importantly, we can compute the jacobian of the storage space of the object with respect to its tangent space:

In [ ]:
display(lie_vals.storage_D_tangent())

This means that we can use the elements of the object to compute a residual, and then compute the jacobian of such a residual with respect to the tangent space of our values object.

In [ ]:
residual = sf.Matrix(6, 1)
residual[0:3, 0] = lie_vals["rot3"] * lie_vals["sub_vals.vec"]
residual[3:6, 0] = lie_vals["sub_vals.pose3"] * lie_vals["sub_vals.vec"]
display(residual)

In [ ]:
residual_D_tangent = residual.jacobian(lie_vals)
display(residual_D_tangent.shape)
display(residual_D_tangent)